**전달사항**

**1. 마지막 벡터 DB 사용 관련 참고**

**2. pinecone API 키 확인 + 우리가 사용하는 pinecone DB 이름은 adv-db-1**

**3. 메타데이터 관련해서 DB 구조는 노션에 첨부한 이미지 참고**

In [ ]:
!pip install langchain_openai

In [ ]:
!pip install pinecone-client

# 벡터 DB 1 구축

**데이터 및 라이브러리 import**

In [ ]:
from langchain_openai import OpenAIEmbeddings
import pandas as pd
import numpy as np
from pinecone import Pinecone, ServerlessSpec

In [ ]:
# 데이터 불러오기
data_긍정 = pd.read_excel("/content/cluster_good_reviews_건강고민별.xlsx")
data_부정 = pd.read_excel("/content/cluster_bad_reviews_건강고민별.xlsx")

**파인콘 벡터 DB 생성**

In [ ]:
# Pinecone 객체 생성 및 API 키 설정
pc = Pinecone(api_key=api_key)

In [ ]:
# Pinecone index 이름 설정 및 생성
index_name = "adv-db-1"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # 임베딩의 차원 수
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'  # 서버리스 사양에 맞는 클라우드 및 리전 설정
        )
    )
index = pc.Index(name=index_name)

**review 데이터 임베딩**

In [ ]:
# open_ai 임베딩
embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [ ]:
# 데이터 임베딩
def embed_review(data):
    reviews = data.iloc[:,1].to_list()

    # 임베딩 후 타입을 바로 확인합니다
    embeddings = []
    for review in reviews:
        embed = embeddings_model.embed_query(review)
        embeddings.append(np.array(embed).astype(np.float32))

    return embeddings

In [ ]:
# Pinecone에 데이터 업로드
def upload_to_pinecone(data, review_category, batch_size=100):
    embeddings = embed_review(data)
    if not embeddings:
        print("유효한 임베딩이 없어 업로드할 수 없습니다.")
        return

    total_embeddings = len(embeddings)
    print(f"{total_embeddings}개의 {type(embeddings[0])} 타입 임베딩을 업로드합니다.")

    # 데이터를 배치 크기로 나누어 업로드
    for i in range(0, total_embeddings, batch_size):
        batch_embeddings = embeddings[i:i + batch_size]
        batch_data = data.iloc[i:i + batch_size]

        # 카테고리를 ID 접두사로 사용하여 고유 ID 생성
        vectors = [(f"{review_category}-{j}", batch_embeddings[j-i], {
            '브랜드명_제품명': batch_data.iloc[j-i][0],
            '건강고민정보': batch_data.iloc[j-i][2],
            '리뷰 요약': batch_data.iloc[j-i][1],
            '리뷰 종류': review_category
        }) for j in range(i, min(i + batch_size, total_embeddings))]

        index.upsert(vectors=vectors)

In [ ]:
# 카테고리 별로 데이터 업로드
upload_to_pinecone(data_긍정, 'good', batch_size=100)
upload_to_pinecone(data_부정, 'bad', batch_size=100)

8232개의 <class 'numpy.ndarray'> 타입 임베딩을 업로드합니다.
10213개의 <class 'numpy.ndarray'> 타입 임베딩을 업로드합니다.


**벡터 DB 확인**

In [ ]:
query_question = input("조회 할 리뷰 요약 : ")
query_embedding = embeddings_model.embed_query(query_question)
query_embedding_np = np.array(query_embedding).astype(np.float32)
query_embedding_list = query_embedding_np.tolist()

조회 할 리뷰 요약 : 가루가 흘릴 수 있어 조심해야 함


In [ ]:
# Pinecone 쿼리 실행
response = index.query(
    vector=query_embedding_list,  # 쿼리 벡터
    top_k=3,                      # 상위 3개의 결과를 반환
    include_metadata=True,        # 메타데이터 포함
    # filter={
    #     '리뷰 종류': 'good'  # '리뷰 종류' 메타데이터가 'good'인 항목만 검색
    # }
)

In [ ]:
result = response['matches']

In [ ]:
result

[{'id': 'bad-8884',
  'metadata': {'건강고민정보': '장건강_유익균유해균균형도움',
               '리뷰 요약': '가루가 흘릴 수 있어 조심해야 함',
               '리뷰 종류': 'bad',
               '브랜드명_제품명': '서울약사신협_프로바이오 생유산균'},
  'score': 1.00000012,
  'values': []},
 {'id': 'bad-9282',
  'metadata': {'건강고민정보': '치아잇몸&잇몸건강',
               '리뷰 요약': '가루 날림이 있어 조심해야 함',
               '리뷰 종류': 'bad',
               '브랜드명_제품명': '애터미_컬러푸드 비타민C'},
  'score': 0.967417538,
  'values': []},
 {'id': 'bad-9241',
  'metadata': {'건강고민정보': '치아잇몸&잇몸건강',
               '리뷰 요약': '가루가 뭉치는 현상이 있어 조심해서 섭취해야 함',
               '리뷰 종류': 'bad',
               '브랜드명_제품명': '광동제약_비타500 데일리 스틱'},
  'score': 0.934337318,
  'values': []}]

# 벡터 DB 2 구축

**데이터 및 라이브러리 import**

In [ ]:
from langchain_openai import OpenAIEmbeddings
import pandas as pd
import numpy as np
from pinecone import Pinecone, ServerlessSpec

In [ ]:
# 데이터 불러오기
data_긍정 = pd.read_excel("/content/긍정_건강고민_일반특성정보.xlsx")
data_부정 = pd.read_excel("/content/부정_건강고민_일반특성정보.xlsx")

**데이터 전처리**

In [ ]:
# '일반특성정보' 컬럼의 데이터를 '\n' 기준으로 분리하여 새로운 행으로 확장
exploded_data_긍정 = data_긍정.set_index('건강고민')['일반특성정보'].str.split('\n').explode().reset_index()
exploded_data_부정 = data_부정.set_index('건강고민')['일반특성정보'].str.split('\n').explode().reset_index()

**파인콘 벡터 DB 생성**

In [ ]:
# Pinecone 객체 생성 및 API 키 설정
pc = Pinecone(api_key=api_key)

In [ ]:
# Pinecone index 이름 설정 및 생성
index_name = "adv-db-2"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # 임베딩의 차원 수
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'  # 서버리스 사양에 맞는 클라우드 및 리전 설정
        )
    )
index = pc.Index(name=index_name)

**review 데이터 임베딩**

In [ ]:
# open_ai 임베딩
embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [ ]:
# 데이터 임베딩
def embed_review(data):
    reviews = data.iloc[:,1].to_list()

    # 임베딩 후 타입을 바로 확인합니다
    embeddings = []
    for review in reviews:
        embed = embeddings_model.embed_query(review)
        embeddings.append(np.array(embed).astype(np.float32))

    return embeddings

In [ ]:
# Pinecone에 데이터 업로드
def upload_to_pinecone(data, review_category, batch_size=100):
    embeddings = embed_review(data)
    if not embeddings:
        print("유효한 임베딩이 없어 업로드할 수 없습니다.")
        return

    total_embeddings = len(embeddings)
    print(f"{total_embeddings}개의 {type(embeddings[0])} 타입 임베딩을 업로드합니다.")

    # 데이터를 배치 크기로 나누어 업로드
    for i in range(0, total_embeddings, batch_size):
        batch_embeddings = embeddings[i:i + batch_size]
        batch_data = data.iloc[i:i + batch_size]

        # 카테고리를 ID 접두사로 사용하여 고유 ID 생성
        vectors = [(f"{review_category}-{j}", batch_embeddings[j-i], {
            '건강고민정보': batch_data.iloc[j-i][0],
            '일반 특성 정보': batch_data.iloc[j-i][1],
            '리뷰 종류': review_category
        }) for j in range(i, min(i + batch_size, total_embeddings))]

        index.upsert(vectors=vectors)

In [ ]:
# 카테고리 별로 데이터 업로드
upload_to_pinecone(exploded_data_긍정, 'good', batch_size=100)
upload_to_pinecone(exploded_data_부정, 'bad', batch_size=100)

673개의 <class 'numpy.ndarray'> 타입 임베딩을 업로드합니다.
507개의 <class 'numpy.ndarray'> 타입 임베딩을 업로드합니다.


**벡터 DB 확인**

In [ ]:
query_question = input("조회 할 일반 특성 정보 : ")
query_embedding = embeddings_model.embed_query(query_question)
query_embedding_np = np.array(query_embedding).astype(np.float32)
query_embedding_list = query_embedding_np.tolist()

조회 할 일반 특성 정보 : 작은 알약


In [ ]:
# Pinecone 쿼리 실행
response = index.query(
    vector=query_embedding_list,  # 쿼리 벡터
    top_k=3,                      # 상위 3개의 결과를 반환
    include_metadata=True,        # 메타데이터 포함
    # filter={
    #     '리뷰 종류': 'good'  # '리뷰 종류' 메타데이터가 'good'인 항목만 검색
    # }
)

In [ ]:
result = response['matches']

In [ ]:
result

[{'id': 'good-177',
  'metadata': {'건강고민정보': '빈혈_혈액생성', '리뷰 종류': 'good', '일반 특성 정보': '작은 알약 크기'},
  'score': 0.958754241,
  'values': []},
 {'id': 'good-75',
  'metadata': {'건강고민정보': '노화&향산화', '리뷰 종류': 'good', '일반 특성 정보': '작은 알약 크기'},
  'score': 0.958754241,
  'values': []},
 {'id': 'good-414',
  'metadata': {'건강고민정보': '체지방_지방대사촉진', '리뷰 종류': 'good', '일반 특성 정보': '작은 알약 크기'},
  'score': 0.958754241,
  'values': []}]

# 벡터 DB 1과 2 합쳐서 DB 3 구축

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from scipy.spatial.distance import cosine

# 모델과 토크나이저 로딩
model_name = "klue/bert-base"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Pinecone index 이름 설정 및 생성
index1 = pc.Index(name="adv-db-1")
index2 = pc.Index(name="adv-db-2")

Bert 모델을 통해 의미 기반 유사도 측정

In [ ]:
def calculate_cosine_similarity(text1, text2):
    # 텍스트를 토큰화 및 입력 형식으로 변환
    inputs1 = tokenizer(text1, return_tensors="pt")
    inputs2 = tokenizer(text2, return_tensors="pt")

    # 모델을 통해 임베딩 생성
    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)

    # [CLS] 토큰의 임베딩 추출
    embeddings1 = outputs1.last_hidden_state[:, 0, :]
    embeddings2 = outputs2.last_hidden_state[:, 0, :]

    # 임베딩을 1차원 벡터로 변환
    embeddings1 = embeddings1.squeeze().numpy()
    embeddings2 = embeddings2.squeeze().numpy()

    # 코사인 유사도 계산
    similarity = 1 - cosine(embeddings1, embeddings2)

    return similarity

good 리뷰에 대하여 벡터 DB1에 DB2의 정보 추가하기

In [ ]:
# 각 idx에 대해 반복 실행
for i in range(11):  # good : 8232 존재
    idx = f'good-{i}'

    # ID가 idx인 데이터를 조회합니다
    result = index1.fetch(ids=[idx])

    # 메타데이터와 벡터 데이터 추출
    metadata = result['vectors'][idx]['metadata']
    text1 = metadata['리뷰 요약']
    vector_values = result['vectors'][idx]['values']
    concern_category = metadata['건강고민정보']

    # DB2에서 '건강고민정보' metadata가 concern_category와 일치하는 데이터를 검색합니다
    query_result = index2.query(
        vector=vector_values,
        top_k=1,                      # 상위 1개의 결과를 반환
        include_metadata=True,        # 메타데이터 포함
        filter={
            '건강고민정보': concern_category,
            '리뷰 종류': 'good'
        }
    )

    # matches 리스트가 비어있지 않은지 확인하고 '일반 특성 정보'와 'score' 추출
    if query_result['matches']:
        text2 = query_result['matches'][0]['metadata']['일반 특성 정보']
        score = query_result['matches'][0]['score']
    else:
        text2 = ''  # matches 리스트가 비어 있을 경우 빈 문자열 할당
        score = 0   # matches 리스트가 비어 있을 경우 0 할당

    similarity = calculate_cosine_similarity(text1, text2)

    # score와 similarity에 따라 is_general 값을 설정   ->   similarity가 0.75, score가 0.85 보다 크면 연관있다고 판단
    if score > 0.85 and similarity > 0.75:
        metadata['is_general'] = 'True'
        metadata['general_noun'] = text2
    else:
        metadata['is_general'] = 'False'
        metadata['general_noun'] = ''

    # 변경된 메타데이터와 기존 벡터를 사용하여 데이터 업데이트
    index1.upsert(vectors=[(idx, vector_values, metadata)])

    # print(f"Updated {idx} with is_general={metadata['is_general']} based on score={score:.2f} and similarity={similarity:.2f}.")

bad 리뷰에 대하여 벡터 DB1에 DB2의 정보 추가하기

In [ ]:
# 각 idx에 대해 반복 실행
for i in range(10212):  # bad : 10213 존재
    idx = f'bad-{i}'

    # ID가 idx인 데이터를 조회합니다
    result = index1.fetch(ids=[idx])

    # 메타데이터와 벡터 데이터 추출
    metadata = result['vectors'][idx]['metadata']
    text1 = metadata['리뷰 요약']
    vector_values = result['vectors'][idx]['values']
    concern_category = metadata['건강고민정보']

    # DB2에서 '건강고민정보' metadata가 concern_category와 일치하는 데이터를 검색합니다
    query_result = index2.query(
        vector=vector_values,
        top_k=1,                      # 상위 1개의 결과를 반환
        include_metadata=True,        # 메타데이터 포함
        filter={
            '건강고민정보': concern_category,
            '리뷰 종류': 'bad'
        }
    )

    # matches 리스트가 비어있지 않은지 확인하고 '일반 특성 정보'와 'score' 추출
    if query_result['matches']:
        text2 = query_result['matches'][0]['metadata']['일반 특성 정보']
        score = query_result['matches'][0]['score']
    else:
        text2 = ''  # matches 리스트가 비어 있을 경우 빈 문자열 할당
        score = 0   # matches 리스트가 비어 있을 경우 0 할당

    similarity = calculate_cosine_similarity(text1, text2)

    # score와 similarity에 따라 is_general 값을 설정   ->   similarity가 0.75, score가 0.85 보다 크면 연관있다고 판단
    if score > 0.85 and similarity > 0.75:
        metadata['is_general'] = 'True'
        metadata['general_noun'] = text2
    else:
        metadata['is_general'] = 'False'
        metadata['general_noun'] = ''

    # 변경된 메타데이터와 기존 벡터를 사용하여 데이터 업데이트
    index1.upsert(vectors=[(idx, vector_values, metadata)])

    # print(f"Updated {idx} with is_general={metadata['is_general']} based on score={score:.2f} and similarity={similarity:.2f}.")

# 벡터 DB 사용 관련

In [ ]:
from langchain_openai import OpenAIEmbeddings
import pandas as pd
import numpy as np
from pinecone import Pinecone, ServerlessSpec

In [ ]:
# Pinecone 객체 생성 및 API 키 설정
pc = Pinecone(api_key=api_key)

In [ ]:
# Pinecone index 이름 설정 및 생성
index_name = "adv-db-1"
index = pc.Index(name=index_name)

In [ ]:
query_question = input("조회 할 리뷰 요약 : ")
query_embedding = embeddings_model.embed_query(query_question)
query_embedding_np = np.array(query_embedding).astype(np.float32)
query_embedding_list = query_embedding_np.tolist()

In [ ]:
# Pinecone 쿼리 실행
response = index.query(
    vector=query_embedding_list,  # 쿼리 벡터
    top_k=3,                      # 상위 3개의 결과를 반환
    include_metadata=True,        # 메타데이터 포함
    filter={
        '건강고민정보': concern_category, # 메타데이터 필터
        '리뷰 종류': 'good'  # '리뷰 종류' 메타데이터가 'good'인 항목만 검색
    }
)

In [ ]:
result = response['matches']

In [ ]:
result